In [1]:
import numpy as np
import psycopg2
from psycopg2 import OperationalError
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)
psycopg2.extensions.register_adapter(np.bool_, psycopg2._psycopg.AsIs)
from pytrends.request import TrendReq
from time import sleep
import pandas as pd
import seaborn as sns
import json

pytrends = TrendReq(hl='en-US', tz=360)

In [2]:
#get credentials
with open("postgre_credentials.json") as file:
    data = json.load(file)

In [3]:
def reset_connection():
    if 'connection' not in locals():
        connection = psycopg2.connect(
        database = data["db_name"], 
        user = data["db_user"],
        password = data["db_password"],
        host = data["db_host"],
        port = data["db_port"]
        )
        cur = connection.cursor()
        return connection,cur;
    else:
        connection.close()
        connection = psycopg2.connect(
        database = data["db_name"], 
        user = data["db_user"],
        password = data["db_password"],
        host = data["db_host"],
        port = data["db_port"]
        )
        cur = connection.cursor()
        return connection, cur;

In [4]:
connection, cur = reset_connection()

In [5]:
query="""SELECT eng_name, alternate_name
FROM anime join show_name ON anime.anime_id = show_name.anime_id
WHERE air_status LIKE 'Finished Airing'
"""
cur.execute(query)
anime_ids = cur.fetchall()

In [6]:
#anime_ids

In [110]:
#input country string - Example: 'US'
#input show_tuple - tuple with eng_name as the first entry. May contain 'NA'
#output - google trends_table is filled with show info with show_name attached
def call_google_trends(country, show_tuple):    
    try:
        name = "fill"
        for entry in show_tuple:
            if entry != "NA":
                name = entry
                break       
        if name!= "fill":
            pytrends.build_payload(kw_list=[name],cat=0,timeframe='all', geo=country)
            data = pytrends.interest_over_time()
            #print(data)
            if not data.empty:
                data = data.drop(labels=['isPartial'],axis='columns')
                return data
        sleep(3)
    except (RuntimeError):
        sleep(3)
        print("Invalid Name?")
        print(RuntimeError)

In [115]:
def insert_pop_score(show_df):
    #show_df.index = np.arange(1,205)
    show_name = show_df.columns[0]
    cur.execute("""SELECT show_name_id
                FROM show_name
                WHERE alternate_name LIKE %s""",
                (show_name,))
    show_name_id = cur.fetchone()[0]
    
    #Does the show already exists in google_trends table?
    cur.execute("""SELECT show_name
                FROM google_trends
                WHERE show_name = %s""",(show_name_id,))
    is_in_google_trends = cur.fetchone()

    if is_in_google_trends is None:
        #print(is_in_google_trends)
        for row in show_df.iterrows():
            popularity_score = row[1][0]
            date_id = row[0]
            cur.execute("""INSERT INTO google_trends(show_name, date, popularity_score)
                        VALUES (%s, %s, %s)
                        """,(show_name_id, date_id, popularity_score))
            connection.commit()
    sleep(7)
    





In [9]:

anime_ids = [entry for entry in anime_ids if entry[0]!='NA']

In [108]:
connection.close()
connection, cur = reset_connection()

In [118]:
#anime_df_test = call_google_trends('US', anime_ids[25])
#anime_df_test.index = np.arange(1,205)
# insert_pop_score(anime_df_test)

In [119]:

for anime in anime_ids:
    try:
        anime_df = call_google_trends('US',anime)
        if anime_df is not None:
            anime_df.index = np.arange(1,205)
            insert_pop_score(anime_df)
        connection.close()
        connection, cur = reset_connection()
    except:
        continue
connection.close()

ResponseError: The request failed: Google returned a response with code 400.

In [230]:
connection.close()